In [1]:
import random

In [2]:
class Damage:
    def __init__(self, rolls, flat_bonus):
        self.rolls = rolls
        self.flat_bonus = flat_bonus
    def roll(self):
        rolls = sum([random.randint(1, roll) for roll in self.rolls])
        return rolls + self.flat_bonus

In [3]:
class Attack:
    def __init__(self, to_hit, damage):
        self.to_hit = to_hit
        self.damage = damage
    def roll_hit(self, advantage = False, disadvantage = False):
        if advantage and disadvantage:
            advantage = disadvantage = False
        roll1 = self.to_hit + random.randint(1, 21)
        if advantage:
            roll2 = self.to_hit + random.randint(1, 21)
            return max(roll1, roll2), self.damage()
        if disadvantage:
            roll2 = self.to_hit + random.randint(1, 21)
            return min(roll1, roll2), self.damage()
        return roll1, self.damage.roll()

In [4]:
class Character:
    def __init__(self, hp, ac, attacks, stats, name):
        self.hp = hp
        self.max_hp = hp
        self.ac = ac
        if not isinstance(attacks, list):
            attacks = [attacks]
        self.attacks = attacks
        self.stats = stats
        self.name = name
    def roll_init(self):
        return self.stats['dex'] + random.randint(1, 21) + self.stats['dex']/100.
    def take_hit(self, to_hit, damage):
        if to_hit >= self.ac:
            self.hp -= damage

In [29]:
fighter = Character(
    hp = 13, ac = 18, attacks = Attack(to_hit = 4, damage = Damage(rolls = [6], flat_bonus = 2)), stats = {'dex':-1},
    name = 'fighter')
cleric = Character(
    hp = 10, ac = 18, attacks = Attack(to_hit = 5, damage = Damage(rolls = [8], flat_bonus = 5)), stats = {'dex':-1},
    name = 'cleric')
warlock = Character(
    hp = 11, ac = 15, attacks = Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 3)), stats = {'dex':2},
    name = 'warlock')
rogue = Character(
    hp = 11, ac = 14, attacks = [
        Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 3)),
        Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 0))
    ], stats = {'dex':3}, name = 'rogue')
fighter2 = Character(
    hp = 13, ac = 18, attacks = Attack(to_hit = 4, damage = Damage(rolls = [6], flat_bonus = 2)), stats = {'dex':-1},
    name = 'fighter')
cleric2 = Character(
    hp = 10, ac = 18, attacks = Attack(to_hit = 5, damage = Damage(rolls = [8], flat_bonus = 5)), stats = {'dex':-1},
    name = 'cleric')
warlock2 = Character(
    hp = 11, ac = 15, attacks = Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 3)), stats = {'dex':2},
    name = 'warlock')
rogue2 = Character(
    hp = 11, ac = 14, attacks = [
        Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 3)),
        Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 0))
    ], stats = {'dex':3}, name = 'rogue')

team_PCs = [fighter, cleric, rogue, warlock]
team_NPCs = [fighter2, cleric2, warlock2, rogue2]

In [30]:
import numpy as np

In [31]:
def roll_init(x):
    init = [xi.roll_init() for xi in x]
    order = np.argsort(init)
    init_order = [x[i] for i in order[::-1]]
    return init_order
    

In [32]:
def combat(pcs, npcs):
    all_characters = pcs + npcs
    initiative = roll_init(all_characters)
    [setattr(c, 'hp', c.max_hp) for c in initiative]
    for _ in range(100):
        for character in initiative:
            for attack in character.attacks:
                attack, dmg = character.attacks[0].roll_hit()
                if character in pcs:
                    receiver = npcs[0]
    #                 print('www')
                elif character in npcs:
                    receiver = pcs[0]
    #                 print('bbb')
                else:
                    continue
                receiver.take_hit(attack, dmg)
                npcs = [c for c in npcs if c.hp > 0]
                pcs = [c for c in pcs if c.hp > 0]
    #             print(len(npcs), len(pcs))
                if len(npcs) == 0:
                    return 0
                if len(pcs) == 0:
                    return 1
        initiative = [c for c in initiative if c.hp > 0]
    return -1

In [47]:
sum([combat(team_PCs, team_NPCs) for _ in range(10000)])/10000

0.5061